In [1]:
%pip install ipython-autotime
%pip install pandas
%pip install numpy
%load_ext autotime


Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.
time: 0 ns (started: 2023-07-20 15:05:04 +07:00)


In [2]:
import pandas as pd
import numpy as np

time: 766 ms (started: 2023-07-20 15:05:04 +07:00)


# Read dataset in JSON

In [3]:
data = pd.read_json('titanic_json.json')
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,None,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,None,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,None,S


time: 47 ms (started: 2023-07-20 15:05:05 +07:00)


In [4]:
data.shape

(891, 12)

time: 0 ns (started: 2023-07-20 15:05:05 +07:00)


# Preprocessing

Kolom yang akan menjadi prediktor adalah Survived, Pclass, Sex, Age, SibSp, Parch, Fare, Cabin, dan Embarked.

Kolom respon adalah Survived.

Kolom dengan data non-angka akan diubah menjadi angka dengan teknik categorical label encoding.

Sebagai contoh, Sex:male diberi label 1, Sex:female diberi label 2.

Missing value pada kolom kategorik (Pclass, Sex, SibSp, Parch, Cabin, dan Embarked) akan diisi dengan nilai modus.

Missing value pada kolom numerik (Fare, Age) akan diisi dengan median.

#### Pisahkan X dan y

In [5]:
# Pilih variabel X yang sudah dalam bentuk angka
X_numeric = data[['Pclass', 'Age', 'SibSp', 'Parch', 'Fare']]

# Pilih variabel X yang masih bukan angka
X_non_numeric = data[['Sex', 'Cabin', 'Embarked']]

# Pilih variabel Y
y = data['Survived']

time: 15 ms (started: 2023-07-20 15:05:05 +07:00)


### Ubah kolom non-angka ke dalam angka dengan label encoding

Kolom yg ingin kita ubah adalah Sex, Cabin, dan Embarked.

In [6]:
from sklearn.preprocessing import LabelEncoder
X_non_numeric_encode = X_non_numeric.apply(LabelEncoder().fit_transform)
X_non_numeric_encode

,Sex,Cabin,Embarked
0,1,147,2
1,0,81,0
2,0,147,2
3,0,55,2
4,1,147,2
...,...,...,...
886,1,147,2
887,0,30,2
888,0,147,2
889,1,60,0


time: 1.22 s (started: 2023-07-20 15:05:05 +07:00)


In [7]:
# Gabung hasilnya menjadi satu kesatuan yang utuh
X = pd.concat([X_numeric, X_non_numeric_encode], axis = 1)
X

,Pclass,Age,SibSp,Parch,Fare,Sex,Cabin,Embarked
0,3,22.0,1,0,7.2500,1,147,2
1,1,38.0,1,0,71.2833,0,81,0
2,3,26.0,0,0,7.9250,0,147,2
3,1,35.0,1,0,53.1000,0,55,2
4,3,35.0,0,0,8.0500,1,147,2
...,...,...,...,...,...,...,...,...
886,2,27.0,0,0,13.0000,1,147,2
887,1,19.0,0,0,30.0000,0,30,2
888,3,NaN,1,2,23.4500,0,147,2
889,1,26.0,0,0,30.0000,1,60,0


time: 0 ns (started: 2023-07-20 15:05:06 +07:00)


### Observasi ada tidaknya missing value

In [8]:
X.isna().sum()

Pclass        0
Age         177
SibSp         0
Parch         0
Fare          0
Sex           0
Cabin         0
Embarked      0
dtype: int64

time: 31 ms (started: 2023-07-20 15:05:06 +07:00)


### Isi missing value kolom angka (Fare, Age) dengan median

In [9]:
X['Fare'].fillna(X['Fare'].median(), inplace = True)
X['Age'].fillna(X['Age'].median(), inplace = True)
X

,Pclass,Age,SibSp,Parch,Fare,Sex,Cabin,Embarked
0,3,22.0,1,0,7.2500,1,147,2
1,1,38.0,1,0,71.2833,0,81,0
2,3,26.0,0,0,7.9250,0,147,2
3,1,35.0,1,0,53.1000,0,55,2
4,3,35.0,0,0,8.0500,1,147,2
...,...,...,...,...,...,...,...,...
886,2,27.0,0,0,13.0000,1,147,2
887,1,19.0,0,0,30.0000,0,30,2
888,3,28.0,1,2,23.4500,0,147,2
889,1,26.0,0,0,30.0000,1,60,0


time: 16 ms (started: 2023-07-20 15:05:06 +07:00)


### Isi missing value kolom kategorik dengan modus

In [10]:
X['Pclass'].fillna(X['Pclass'].mode(), inplace = True)
X['SibSp'].fillna(X['SibSp'].mode(), inplace = True)
X['Parch'].fillna(X['Parch'].mode(), inplace = True)
X['Sex'].fillna(X['Sex'].mode(), inplace = True)
X['Cabin'].fillna(X['Cabin'].mode(), inplace = True)
X['Embarked'].fillna(X['Embarked'].mode(), inplace = True)
X

,Pclass,Age,SibSp,Parch,Fare,Sex,Cabin,Embarked
0,3,22.0,1,0,7.2500,1,147,2
1,1,38.0,1,0,71.2833,0,81,0
2,3,26.0,0,0,7.9250,0,147,2
3,1,35.0,1,0,53.1000,0,55,2
4,3,35.0,0,0,8.0500,1,147,2
...,...,...,...,...,...,...,...,...
886,2,27.0,0,0,13.0000,1,147,2
887,1,19.0,0,0,30.0000,0,30,2
888,3,28.0,1,2,23.4500,0,147,2
889,1,26.0,0,0,30.0000,1,60,0


time: 47 ms (started: 2023-07-20 15:05:06 +07:00)


In [11]:
X.isna().sum()

Pclass      0
Age         0
SibSp       0
Parch       0
Fare        0
Sex         0
Cabin       0
Embarked    0
dtype: int64

time: 15 ms (started: 2023-07-20 15:05:06 +07:00)


# Train Test Split

Setelah data 'bersih', saatnya kita bikin model sesuai kreativitas masing2

In [12]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.23923, random_state = 455518)

time: 109 ms (started: 2023-07-20 15:05:06 +07:00)


# Evaluation Function

In [13]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report
def evaluate_classification(y_true, y_pred):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    print(classification_report(y_true, y_pred))
    return {'Accuracy': accuracy, 'Precision': precision, 'Recall': recall, 'F1-score': f1}

time: 0 ns (started: 2023-07-20 15:05:06 +07:00)


# Model

In [14]:
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

time: 235 ms (started: 2023-07-20 15:05:06 +07:00)


## Support Vector Machine

In [15]:
svm_clf = SVC()
svm_clf.fit(x_train, y_train)
svm_pred = svm_clf.predict(x_test)
evaluate_classification(y_test, svm_pred)

              precision    recall  f1-score   support

           0       0.70      0.96      0.81       134
           1       0.81      0.33      0.46        80

    accuracy                           0.72       214
   macro avg       0.76      0.64      0.64       214
weighted avg       0.74      0.72      0.68       214



{'Accuracy': 0.719626168224299,
 'Precision': 0.8125,
 'Recall': 0.325,
 'F1-score': 0.46428571428571436}

time: 47 ms (started: 2023-07-20 15:05:07 +07:00)


## Decision Tree Classifier

In [16]:
dt_clf = DecisionTreeClassifier(max_depth=5)
dt_clf.fit(x_train, y_train)
dt_pred = dt_clf.predict(x_test)
evaluate_classification(y_test, dt_pred)

              precision    recall  f1-score   support

           0       0.82      0.95      0.88       134
           1       0.88      0.65      0.75        80

    accuracy                           0.84       214
   macro avg       0.85      0.80      0.81       214
weighted avg       0.84      0.84      0.83       214



{'Accuracy': 0.8364485981308412,
 'Precision': 0.8813559322033898,
 'Recall': 0.65,
 'F1-score': 0.7482014388489208}

time: 16 ms (started: 2023-07-20 15:05:07 +07:00)


# Random Forest

In [17]:
rf_clf = RandomForestClassifier(n_estimators=100, max_depth=5)
rf_clf.fit(x_train, y_train)
rf_pred = rf_clf.predict(x_test)
evaluate_classification(y_test, rf_pred)

              precision    recall  f1-score   support

           0       0.80      0.95      0.87       134
           1       0.87      0.60      0.71        80

    accuracy                           0.82       214
   macro avg       0.84      0.77      0.79       214
weighted avg       0.83      0.82      0.81       214



{'Accuracy': 0.8177570093457944,
 'Precision': 0.8727272727272727,
 'Recall': 0.6,
 'F1-score': 0.711111111111111}

time: 250 ms (started: 2023-07-20 15:05:07 +07:00)


# All Model

In [18]:
# Model
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neural_network import MLPClassifier

# Metrics evaluasi
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

def auto_model(x_kereta, y_kereta, x_ujian, y_ujian):
  daftar_model_kita = [SVC(), DecisionTreeClassifier(), RandomForestClassifier()]

  akurasi = []
  F1_score = []
  precission = []
  Recall = []
 
  for model in daftar_model_kita:
    model.fit(x_kereta, y_kereta)
    prediksi_model_kita = model.predict(x_ujian)
    akurasi.append(accuracy_score(y_true = y_ujian, y_pred = prediksi_model_kita))
    F1_score.append(f1_score(y_true = y_ujian, y_pred = prediksi_model_kita))
    precission.append(precision_score(y_true = y_ujian, y_pred = prediksi_model_kita))
    Recall.append(recall_score(y_true = y_ujian, y_pred = prediksi_model_kita))

  # Jadiin sebuah dataframe biar keren gituloh
  hasil = {
    'model' : daftar_model_kita,
    'akurasi' : akurasi,
    'F1_score' : F1_score,
    'precission' : precission,
    'Recall' : Recall
  }
  return(pd.DataFrame(hasil))

time: 78 ms (started: 2023-07-20 15:05:07 +07:00)


In [19]:
# Saatnya kita uji
Auto_Model = auto_model(x_train, y_train, x_test, y_test)

time: 359 ms (started: 2023-07-20 15:05:07 +07:00)


In [20]:
Auto_Model.sort_values(by='akurasi', ascending=False)

,model,akurasi,F1_score,precission,Recall
2,"(DecisionTreeClassifier(max_features='sqrt', r...",0.827103,0.754967,0.802817,0.7125
1,DecisionTreeClassifier(),0.813084,0.726027,0.803030,0.6625
0,SVC(),0.719626,0.464286,0.812500,0.3250


time: 172 ms (started: 2023-07-20 15:05:08 +07:00)
